In [1]:
import sys
print(sys.executable)
print(sys.version)


c:\Users\Tishie\Desktop\Ambulance-Allocation-Research\venv\Scripts\python.exe
3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]


In [4]:
import pandas as pd 

df = pd.read_csv("../data/district_flood_data.csv")
df.head()


,District,Year,Flood_event_count,Affected_population,Severity_index,avg_rainfall_mm,Road_disruption_level,Ambulances needed
0,Kandy,2010,2,300,3,150,3,98
1,Kandy,2011,2,340,3,160,3,5
2,Kandy,2012,2,380,3,170,3,5
3,Kandy,2013,1,280,2,140,2,3
4,Kandy,2014,2,350,3,160,3,5


In [5]:
df.info()
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   District               80 non-null     object
 1   Year                   80 non-null     int64 
 2   Flood_event_count      80 non-null     int64 
 3   Affected_population    80 non-null     int64 
 4   Severity_index         80 non-null     int64 
 5   avg_rainfall_mm        80 non-null     int64 
 6   Road_disruption_level  80 non-null     int64 
 7   Ambulances needed      80 non-null     int64 
dtypes: int64(7), object(1)
memory usage: 5.1+ KB


District                 0
Year                     0
Flood_event_count        0
Affected_population      0
Severity_index           0
avg_rainfall_mm          0
Road_disruption_level    0
Ambulances needed        0
dtype: int64

In [8]:
X = df[
    [ 
        "Flood_event_count",
        "Affected_population",
        "Severity_index",
        "avg_rainfall_mm",
        "Road_disruption_level"
    ]
]

Y = df["Ambulances needed"]

In [10]:
X.head(), Y.head()

(   Flood_event_count  Affected_population  Severity_index  avg_rainfall_mm  \
 0                  2                  300               3              150   
 1                  2                  340               3              160   
 2                  2                  380               3              170   
 3                  1                  280               2              140   
 4                  2                  350               3              160   
 
    Road_disruption_level  
 0                      3  
 1                      3  
 2                      3  
 3                      2  
 4                      3  ,
 0    98
 1     5
 2     5
 3     3
 4     5
 Name: Ambulances needed, dtype: int64)

In [11]:
df.columns


Index(['District', 'Year', 'Flood_event_count', 'Affected_population',
       'Severity_index', 'avg_rainfall_mm', 'Road_disruption_level',
       'Ambulances needed'],
      dtype='object')

In [12]:
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)


In [13]:
df.columns


Index(['district', 'year', 'flood_event_count', 'affected_population',
       'severity_index', 'avg_rainfall_mm', 'road_disruption_level',
       'ambulances_needed'],
      dtype='object')

In [14]:
df.isna().sum()

district                 0
year                     0
flood_event_count        0
affected_population      0
severity_index           0
avg_rainfall_mm          0
road_disruption_level    0
ambulances_needed        0
dtype: int64

In [15]:
df["ambulances_needed"].describe()


count    80.000000
mean     11.337500
std      10.667523
min       3.000000
25%       7.000000
50%      10.000000
75%      13.250000
max      98.000000
Name: ambulances_needed, dtype: float64

In [16]:
df[df["ambulances_needed"] > 25]


,district,year,flood_event_count,affected_population,severity_index,avg_rainfall_mm,road_disruption_level,ambulances_needed
0,Kandy,2010,2,300,3,150,3,98


In [17]:
df.loc[df["ambulances_needed"] > 25, "ambulances_needed"] = 5


In [18]:
df["ambulances_needed"].describe()


count    80.000000
mean     10.175000
std       4.227023
min       3.000000
25%       7.000000
50%       9.500000
75%      13.000000
max      20.000000
Name: ambulances_needed, dtype: float64

In [19]:
X = df[
    [
        "flood_event_count",
        "affected_population",
        "severity_index",
        "avg_rainfall_mm",
        "road_disruption_level"
    ]
]

y = df["ambulances_needed"]


In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train.shape, X_test.shape


((64, 5), (16, 5))

In [21]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

lr = LinearRegression()
lr.fit(X_train, y_train)

lr_preds = lr.predict(X_test)
lr_mae = mean_absolute_error(y_test, lr_preds)

print("Linear Regression MAE:", lr_mae)


Linear Regression MAE: 0.9530283620273245


In [22]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

rf = RandomForestRegressor(
    n_estimators=200,
    max_depth=5,
    min_samples_leaf=3,
    random_state=42
)

rf.fit(X_train, y_train)

rf_preds = rf.predict(X_test)
rf_mae = mean_absolute_error(y_test, rf_preds)

print("Random Forest MAE:", rf_mae)


Random Forest MAE: 0.2810940769300143


In [23]:
import pandas as pd

feature_importance = pd.Series(
    rf.feature_importances_,
    index=X.columns
).sort_values(ascending=False)

feature_importance


affected_population      0.476484
flood_event_count        0.219375
avg_rainfall_mm          0.144060
road_disruption_level    0.080273
severity_index           0.079808
dtype: float64

In [24]:
def predict_district_ambulances(
    flood_event_count,
    affected_population,
    severity_index,
    avg_rainfall_mm,
    road_disruption_level
):
    input_data = pd.DataFrame([{
        "flood_event_count": flood_event_count,
        "affected_population": affected_population,
        "severity_index": severity_index,
        "avg_rainfall_mm": avg_rainfall_mm,
        "road_disruption_level": road_disruption_level
    }])
    
    return rf.predict(input_data)[0]


In [25]:
predict_district_ambulances(
    flood_event_count=4,
    affected_population=900,
    severity_index=5,
    avg_rainfall_mm=270,
    road_disruption_level=5
)


np.float64(12.866023809523814)

In [26]:
df["predicted_ambulances"] = rf.predict(X)


In [27]:
df[["district", "year", "ambulances_needed", "predicted_ambulances"]].head(10)


,district,year,ambulances_needed,predicted_ambulances
0,Kandy,2010,5,4.218655
1,Kandy,2011,5,4.748931
2,Kandy,2012,5,5.371055
3,Kandy,2013,3,4.113072
4,Kandy,2014,5,4.770681
5,Kandy,2015,6,5.941875
6,Kandy,2016,8,8.026414
7,Kandy,2017,9,8.736701
8,Kandy,2018,6,6.190086
9,Kandy,2019,9,8.916476


In [28]:
district_max = (
    df.groupby("district")["predicted_ambulances"]
    .max()
    .round(0)
)

district_max


district
Galle           17.0
Gampaha         18.0
Kandy           13.0
Nuwara Eliya    13.0
Ratnapura       18.0
Name: predicted_ambulances, dtype: float64

In [29]:
df[
    (df["district"] == "Kandy")
][["year", "predicted_ambulances"]].sort_values(
    by="predicted_ambulances", ascending=False
)


,year,predicted_ambulances
15,2025,13.387207
14,2024,13.281181
11,2021,12.767690
13,2023,10.094763
12,2022,9.888613
9,2019,8.916476
7,2017,8.736701
6,2016,8.026414
8,2018,6.190086
5,2015,5.941875


In [30]:
import numpy as np

df["predicted_ambulances_rounded"] = np.ceil(
    df["predicted_ambulances"]
).astype(int)

df[["district", "year", "predicted_ambulances", "predicted_ambulances_rounded"]].head()


,district,year,predicted_ambulances,predicted_ambulances_rounded
0,Kandy,2010,4.218655,5
1,Kandy,2011,4.748931,5
2,Kandy,2012,5.371055,6
3,Kandy,2013,4.113072,5
4,Kandy,2014,4.770681,5


In [31]:
district_final = (
    df.groupby("district")["predicted_ambulances_rounded"]
    .max()
)

district_final


district
Galle           17
Gampaha         18
Kandy           14
Nuwara Eliya    14
Ratnapura       19
Name: predicted_ambulances_rounded, dtype: int64